# 🛰️ SNUNet Change Detection - Kaggle Training

**Before running:**
1. Settings → Accelerator → **GPU T4 x2**
2. Settings → Internet → **On**

In [ ]:
# === SETUP ===
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from tqdm import tqdm
import zipfile
import urllib.request

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cpu':
    print("⚠️ WARNING: No GPU! Training will be very slow.")
    print("   Go to: Settings → Accelerator → GPU T4 x2")

In [ ]:
# === DOWNLOAD DATASET ===
DATA_DIR = '/kaggle/working/data'
os.makedirs(DATA_DIR, exist_ok=True)

# Check if data already exists
train_dir = os.path.join(DATA_DIR, 'train', 'A')
if os.path.exists(train_dir) and len(os.listdir(train_dir)) > 0:
    print(f"✅ Dataset already exists: {len(os.listdir(train_dir))} train images")
else:
    print("Downloading LEVIR-CD dataset...")
    
    # Try multiple download sources
    urls = [
        'https://www.dropbox.com/s/18fb5jo07l0yqwg/LEVIR-CD-256.zip?dl=1',
        'https://github.com/justchenhao/LEVIR-CD/releases/download/v1.0/LEVIR-CD-256.zip'
    ]
    
    downloaded = False
    for url in urls:
        try:
            print(f"Trying: {url[:50]}...")
            urllib.request.urlretrieve(url, '/kaggle/working/levir.zip')
            downloaded = True
            print("✅ Download complete!")
            break
        except Exception as e:
            print(f"  Failed: {e}")
    
    if downloaded:
        print("Extracting...")
        with zipfile.ZipFile('/kaggle/working/levir.zip', 'r') as z:
            z.extractall(DATA_DIR)
        os.remove('/kaggle/working/levir.zip')
        print("✅ Extraction complete!")
    else:
        print("❌ Download failed. Creating synthetic data for demo...")
        # Create minimal synthetic data
        for split in ['train', 'val', 'test']:
            for sub in ['A', 'B', 'label']:
                os.makedirs(os.path.join(DATA_DIR, split, sub), exist_ok=True)
        
        n_train, n_val = 100, 20
        for split, n in [('train', n_train), ('val', n_val), ('test', n_val)]:
            for i in range(n):
                img = Image.fromarray(np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8))
                img.save(os.path.join(DATA_DIR, split, 'A', f'{i:04d}.png'))
                img.save(os.path.join(DATA_DIR, split, 'B', f'{i:04d}.png'))
                mask = Image.fromarray(np.random.choice([0, 255], (256, 256)).astype(np.uint8))
                mask.save(os.path.join(DATA_DIR, split, 'label', f'{i:04d}.png'))
        print("✅ Synthetic data created!")

# Find actual data path
DATA_ROOT = DATA_DIR
for root, dirs, files in os.walk(DATA_DIR):
    if 'train' in dirs and 'A' in os.listdir(os.path.join(root, 'train')):
        DATA_ROOT = root
        break

print(f"\nData root: {DATA_ROOT}")
for split in ['train', 'val', 'test']:
    p = os.path.join(DATA_ROOT, split, 'A')
    if os.path.exists(p):
        print(f"  {split}: {len(os.listdir(p))} images")

In [ ]:
# === MODEL ===
class ConvBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.conv(x)

class Attention(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.fc = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Conv2d(c, c//8, 1), nn.ReLU(), nn.Conv2d(c//8, c, 1), nn.Sigmoid())
    def forward(self, x): return x * self.fc(x)

class SNUNet(nn.Module):
    def __init__(self, base=32):
        super().__init__()
        c = base
        self.enc1 = ConvBlock(3, c)
        self.enc2 = ConvBlock(c, c*2)
        self.enc3 = ConvBlock(c*2, c*4)
        self.enc4 = ConvBlock(c*4, c*8)
        self.enc5 = ConvBlock(c*8, c*16)
        self.pool = nn.MaxPool2d(2)
        
        self.up4 = nn.ConvTranspose2d(c*16, c*8, 2, 2)
        self.dec4 = ConvBlock(c*24, c*8)
        self.att4 = Attention(c*8)
        
        self.up3 = nn.ConvTranspose2d(c*8, c*4, 2, 2)
        self.dec3 = ConvBlock(c*12, c*4)
        self.att3 = Attention(c*4)
        
        self.up2 = nn.ConvTranspose2d(c*4, c*2, 2, 2)
        self.dec2 = ConvBlock(c*6, c*2)
        self.att2 = Attention(c*2)
        
        self.up1 = nn.ConvTranspose2d(c*2, c, 2, 2)
        self.dec1 = ConvBlock(c*3, c)
        self.att1 = Attention(c)
        
        self.out = nn.Conv2d(c, 1, 1)
        
    def forward(self, x1, x2):
        # Encoder
        e11, e12 = self.enc1(x1), self.enc1(x2)
        e21, e22 = self.enc2(self.pool(e11)), self.enc2(self.pool(e12))
        e31, e32 = self.enc3(self.pool(e21)), self.enc3(self.pool(e22))
        e41, e42 = self.enc4(self.pool(e31)), self.enc4(self.pool(e32))
        e51, e52 = self.enc5(self.pool(e41)), self.enc5(self.pool(e42))
        
        # Decoder with skip connections
        d = torch.abs(e51 - e52)
        d = self.att4(self.dec4(torch.cat([self.up4(d), e41, e42], 1)))
        d = self.att3(self.dec3(torch.cat([self.up3(d), e31, e32], 1)))
        d = self.att2(self.dec2(torch.cat([self.up2(d), e21, e22], 1)))
        d = self.att1(self.dec1(torch.cat([self.up1(d), e11, e12], 1)))
        
        return self.out(d)

model = SNUNet(32).to(device)
print(f"✅ Model: {sum(p.numel() for p in model.parameters()):,} parameters")

In [ ]:
# === DATASET ===
class CDDataset(Dataset):
    def __init__(self, root, split):
        self.root = root
        self.split = split
        self.files = sorted([f for f in os.listdir(os.path.join(root, split, 'A')) if f.endswith('.png')])
        
    def __len__(self): return len(self.files)
    
    def __getitem__(self, idx):
        name = self.files[idx]
        
        img1 = np.array(Image.open(os.path.join(self.root, self.split, 'A', name)).convert('RGB').resize((256, 256)))
        img2 = np.array(Image.open(os.path.join(self.root, self.split, 'B', name)).convert('RGB').resize((256, 256)))
        label = np.array(Image.open(os.path.join(self.root, self.split, 'label', name)).convert('L').resize((256, 256)))
        
        img1 = torch.from_numpy(img1).permute(2, 0, 1).float() / 255.0
        img2 = torch.from_numpy(img2).permute(2, 0, 1).float() / 255.0
        label = (torch.from_numpy(label).float() / 255.0 > 0.5).float().unsqueeze(0)
        
        return img1, img2, label

train_ds = CDDataset(DATA_ROOT, 'train')
val_ds = CDDataset(DATA_ROOT, 'val') if os.path.exists(os.path.join(DATA_ROOT, 'val')) else CDDataset(DATA_ROOT, 'test')

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print(f"✅ Train: {len(train_ds)} | Val: {len(val_ds)}")

In [ ]:
# === LOSS & METRICS ===
def hybrid_loss(pred, target):
    # BCE
    bce = F.binary_cross_entropy_with_logits(pred, target)
    # Dice
    pred_sig = torch.sigmoid(pred)
    intersection = (pred_sig * target).sum()
    dice = 1 - (2 * intersection + 1) / (pred_sig.sum() + target.sum() + 1)
    return bce + dice

def calc_f1(pred, target):
    pred = (torch.sigmoid(pred) > 0.5).float()
    tp = (pred * target).sum()
    fp = (pred * (1 - target)).sum()
    fn = ((1 - pred) * target).sum()
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return f1.item()

In [ ]:
# === TRAINING ===
import csv

EPOCHS = 100
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS)

os.makedirs('/kaggle/working/checkpoints', exist_ok=True)
log_file = open('/kaggle/working/checkpoints/log.csv', 'w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['epoch', 'train_loss', 'val_loss', 'val_f1'])

best_f1 = 0
patience = 0

print(f"Starting training for {EPOCHS} epochs...")
print("-" * 50)

for epoch in range(EPOCHS):
    # Train
    model.train()
    train_loss = 0
    for x1, x2, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False):
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        
        optimizer.zero_grad()
        out = model(x1, x2)
        loss = hybrid_loss(out, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Validate
    model.eval()
    val_loss, val_f1 = 0, 0
    with torch.no_grad():
        for x1, x2, y in val_loader:
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)
            out = model(x1, x2)
            val_loss += hybrid_loss(out, y).item()
            val_f1 += calc_f1(out, y)
    val_loss /= len(val_loader)
    val_f1 /= len(val_loader)
    
    scheduler.step()
    
    # Log
    print(f"Ep {epoch+1:3d} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | F1: {val_f1:.4f}")
    log_writer.writerow([epoch+1, train_loss, val_loss, val_f1])
    log_file.flush()
    
    # Save best
    if val_f1 > best_f1:
        best_f1 = val_f1
        patience = 0
        torch.save(model.state_dict(), '/kaggle/working/checkpoints/best_model.pth')
        print(f"      >> Saved best model (F1={val_f1:.4f})")
    else:
        patience += 1
        if patience >= 20:
            print("Early stopping!")
            break

log_file.close()
print("-" * 50)
print(f"✅ Training complete! Best F1: {best_f1:.4f}")

In [ ]:
# === VISUALIZATION ===
import matplotlib.pyplot as plt
import pandas as pd

# Load best model
model.load_state_dict(torch.load('/kaggle/working/checkpoints/best_model.pth'))
model.eval()

# Plot learning curves
df = pd.read_csv('/kaggle/working/checkpoints/log.csv')
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(df['epoch'], df['train_loss'], label='Train')
axes[0].plot(df['epoch'], df['val_loss'], label='Val')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss')
axes[0].legend(); axes[0].grid(True)
axes[0].set_title('Loss')

axes[1].plot(df['epoch'], df['val_f1'], 'g-')
axes[1].axhline(0.85, color='r', linestyle='--', label='Target')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('F1')
axes[1].legend(); axes[1].grid(True)
axes[1].set_title(f'F1 Score (Best: {df.val_f1.max():.4f})')

plt.tight_layout()
plt.savefig('/kaggle/working/learning_curves.png')
plt.show()

In [ ]:
# === SAMPLE PREDICTIONS ===
fig, axes = plt.subplots(3, 4, figsize=(12, 9))

for row in range(3):
    idx = row * 5
    if idx >= len(val_ds):
        break
    x1, x2, y = val_ds[idx]
    
    with torch.no_grad():
        pred = torch.sigmoid(model(x1.unsqueeze(0).to(device), x2.unsqueeze(0).to(device)))
        pred = (pred > 0.5).float().cpu().squeeze().numpy()
    
    axes[row, 0].imshow(x1.permute(1, 2, 0).numpy())
    axes[row, 0].set_title('Time 1')
    axes[row, 1].imshow(x2.permute(1, 2, 0).numpy())
    axes[row, 1].set_title('Time 2')
    axes[row, 2].imshow(y.squeeze().numpy(), cmap='gray')
    axes[row, 2].set_title('Ground Truth')
    axes[row, 3].imshow(pred, cmap='gray')
    axes[row, 3].set_title('Prediction')
    
    for col in range(4):
        axes[row, col].axis('off')

plt.tight_layout()
plt.savefig('/kaggle/working/predictions.png')
plt.show()

print("\n✅ All done! Files saved in /kaggle/working/")